In [ ]:
!pip install tldextract
!pip install transformers
!pip install fake-useragent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.0 MB/s eta 0:00:00


In [ ]:
import csv
import ssl
import os
import torch
import ast
from urllib.request import Request, urlopen, URLError, urljoin
from urllib.parse import urlparse
import time
import threading
import queue
from bs4 import BeautifulSoup
from bs4.element import Comment
from pathlib import Path
import re
from datetime import datetime
from queue import PriorityQueue
from matplotlib import pyplot as plt
import tldextract
import hashlib
import requests
from google.colab import files as FILE
import pathlib
import pandas as pd
from fake_useragent import UserAgent
from PIL import Image

from transformers import pipeline
from transformers import CLIPProcessor, CLIPModel
from torch.nn import functional as F


clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
### Script Settings ###
conf = dict(
    dir = '/content/drive/My Drive/KAIST/Spider1/',
    imgsDir = "/content/drive/My Drive/KAIST/Spider1/imgs/",
    keywords = ['Elon Musk', 'Tour Eiffel'],
    labels = ['technology','business','politics', 'history'],
    blacklist = ['youtube.com', 'facebook.com',
                   'twitter.com', 'x.com', 'instagram.com', 'linkedin.com','tiktok.com']
)

###
start_time = int(time.time())
crawled_csv = conf['dir']+'crawler_'+"{}.csv".format(start_time)

In [ ]:
#Delete the Crawler Directory to start from a clean dirctory
if os.path.exists(conf['dir']) is False:
  print("Create Directory")
  os.mkdir(conf['dir'])

if os.path.exists(conf['imgsDir']) is False:
  print("Create Imgs Directory")
  os.mkdir(conf['imgsDir'])

crawler_csv_file = open(crawled_csv, mode='a')
crawlerFile_writer = csv.writer(crawler_csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
crawlerFile_writer.writerow(["timestamp","hash","link","post_text","downloaded_imgs","processed","label","true_label"])
crawler_csv_file.close()

In [ ]:
# def chunk_text(text, max_len=510, stride=490):
#   """Split long text into overlapping chunks based on tokenizer."""
#   tokens = tokenizer.tokenize(text)
#   chunks = []
#   for i in range(0, len(tokens), stride):
#       chunk = tokens[i:i + max_len - 2]  # Reserve space for special tokens
#       if not chunk:
#           continue
#       chunk_text = tokenizer.convert_tokens_to_string(chunk)
#       chunks.append(chunk_text)
#       if len(chunk) < max_len - 2:
#           break
#   return chunks

# def summarize_text(text, max_tokens=500):
#     """Perform light summarization on the input text."""

#     tokenizer = summarizer.tokenizer
#     inputs = tokenizer.encode(text, truncation=True, max_length=1024)
#     truncated_text = tokenizer.decode(inputs, skip_special_tokens=True)

#     summary = summarizer(
#         truncated_text,
#         max_length=max_tokens,
#         min_length=20,
#         do_sample=False
#     )[0]['summary_text']

#     return summary

# def get_label(sentence, mode="default"):
#   if mode == "summarize":
#       summarized = summarize_text(sentence, max_tokens=500)
#       print(summarized)
#       all_texts = [summarized] + conf['labels']

#   elif mode == "chunks":
#       sentence_chunks = chunk_text(sentence)
#       all_texts = sentence_chunks + conf['labels']
#   else:
#       splited_sentence = sentence.split(' ')
#       if len(splited_sentence) > 200 :
#           bert_sentence = ' '.join(splited_sentence[:200])
#       else:
#           bert_sentence = sentence
#       all_texts = [bert_sentence] + conf['labels']


#   inputs = tokenizer.batch_encode_plus(
#       all_texts,
#       return_tensors='pt',
#       padding=True,
#       truncation=True,
#       max_length=512,
#   )

#   input_ids = inputs['input_ids']
#   attention_mask = inputs['attention_mask']
#   with torch.no_grad():
#       output = model(input_ids, attention_mask=attention_mask)[0]  # shape: (num_chunks + num_labels, seq_len, hidden)

#   if mode == "chunks":
#       chunk_outputs = output[:len(sentence_chunks)]
#       sentence_rep = chunk_outputs.mean(dim=1).mean(dim=0, keepdim=True)  # mean over sequence, then over chunks → shape (1, hidden)
#       label_reps = output[len(sentence_chunks):].mean(dim=1)
#   else:
#       sentence_rep = output[:1].mean(dim=1)  # (1, hidden)
#       label_reps = output[1:].mean(dim=1)    # (num_labels, hidden)

#   # now find the labels with the highest cosine similarities
#   similarities = F.cosine_similarity(sentence_rep, label_reps)
#   closest = similarities.argsort(descending=True)
#   top_indices = closest[:2]

#   top_labels = {
#       conf['labels'][i]: similarities[i].item()
#       for i in top_indices
#   }

#   return top_labels

def get_label(sentence, images):
    max_len = 77

    # Prepare texts: sentence + labels
    texts = [sentence] + conf['labels']

    # Use clip_processor with truncation and max_length
    text_inputs = clip_processor(
        text=texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_len
    )

    if images:  # images present
        image_inputs = clip_processor(
            images=images,
            return_tensors="pt"
        )

        with torch.no_grad():
            text_outputs = clip_model.get_text_features(**text_inputs)
            image_outputs = clip_model.get_image_features(**image_inputs)

        image_emb = image_outputs.mean(dim=0, keepdim=True)
        sentence_emb = text_outputs[0:1]
        label_embs = text_outputs[1:]

        combined_emb = (sentence_emb + image_emb) / 2

    else:  # only text
        with torch.no_grad():
            text_outputs = clip_model.get_text_features(**text_inputs)
        combined_emb = text_outputs[0:1]
        label_embs = text_outputs[1:]

    similarities = F.cosine_similarity(combined_emb, label_embs)
    closest = similarities.argsort(descending=True)

    top_indices = closest[:2]

    top_labels = {
        conf['labels'][i]: similarities[i].item()
        for i in top_indices
    }

    return top_labels


import json
from collections import defaultdict

def run(save_path, mode="default"):
  df = pd.read_csv(crawled_csv,index_col=False)
  grouped_links = defaultdict(list)
  for i in range(len(df. index)):
    sentence = df.loc[i, 'post_text']
    images_names = df.loc[i, 'downloaded_imgs']
    images_names = ast.literal_eval(images_names)
    images_paths = [conf['imgsDir'] + name for name in images_names]
    valid_images = []
    for path in images_paths:
        try:
            img = Image.open(path)
            valid_images.append(img)
        except:
            print(f"Skipped file {path}")


    prediction = get_label(sentence, mode=mode)
    top_label = max(prediction, key=prediction.get)
    link = df.loc[i, 'link']
    grouped_links[top_label].append(link)
  with open(save_path, 'w', encoding='utf-8') as f:
      json.dump(grouped_links, f, ensure_ascii=False, indent=4)


In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    if re.match(r"[\n]+", str(element)):
        return False
    return True

def scrape_and_save(link, conf, crawled_csv):
    my_ssl = ssl.create_default_context()
    my_ssl.check_hostname = False
    my_ssl.verify_mode = ssl.CERT_NONE

    timestamp = datetime.timestamp(datetime.now())

    try:
        req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
        response = urlopen(req, context=my_ssl)
        soup = BeautifulSoup(response.read(), "html.parser")

        hash_val = int(hashlib.sha256(link.encode('utf-8')).hexdigest(), 16) % 10**8

        # ----- Image scraping -----
        images = soup.find_all('img')
        downloaded_imgs = []

        for i, image in enumerate(images):
            image_link = None
            for attr in ["data-srcset", "data-src", "data-fallback-src", "src"]:
                if image.get(attr):
                    image_link = image[attr]
                    break

            if image_link:
                try:
                    content = requests.get(image_link).content
                    ext = pathlib.Path(image_link).suffix
                    if ext in [".png", ".jpg"]:
                        file_name = f"{conf['imgsDir']}{hash_val}_{i}{ext}"
                        with open(file_name, "wb") as f:
                            f.write(content)
                        downloaded_imgs.append(f"{hash_val}_{i}{ext}")
                except Exception as e:
                    pass

        # ----- Text scraping -----
        texts = soup.find_all(string=True)
        visible_texts = filter(tag_visible, texts)
        cleaned_texts = [t.strip() for t in visible_texts if len(t.strip().split()) > 5]
        post_text = ' '.join(cleaned_texts)

        if len(post_text.split()) < 6:
            print('Text too short, skipping:', link)
            return

        # ----- Write to CSV -----
        with open(crawled_csv, mode='a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writerow([timestamp, hash_val, link, post_text, downloaded_imgs, "No", "N/A", " "])

        print(f"Done scraping {link}")

    except URLError as e:
        print(f"URL {link} error: {e.reason}")
    except Exception as e:
        print(f"Unexpected error on {link}: {e}")

In [ ]:
blocked_domains = conf['blacklist']


def duckduckgo_search(query, max_results=10, retries=100):
    tries = 0
    results = []
    seen_links = set()

    ua = UserAgent()
    session = requests.Session()

    while tries <= retries and len(results) < max_results:
        try:
            url = "https://duckduckgo.com/html/"
            params = {'q': query, 'kl': 'us-en'}
            headers = {'User-Agent': ua.random}
            response = session.get(url, headers=headers,
                                   params=params, timeout=5)

            soup = BeautifulSoup(response.text, 'html.parser')
            links = soup.find_all(
                'a', attrs={'class': 'result__a'}, href=True)
            if not links:
                raise Exception(
                    "No links found — possibly blocked or unexpected page structure.")

            if len(links) < max_results:
                print(
                    f"Less links than max_results, setting max_results to {len(links)}.")
                max_results = len(links)

            for link in links:
                url = link['href']
                title = link.get_text(strip=True)

                parsed_url = urlparse(url)
                if not parsed_url.scheme or not parsed_url.netloc:
                    continue

                base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"

                if not any(parsed_url.netloc.endswith(domain) for domain in blocked_domains):
                    if url not in seen_links:
                        seen_links.add(url)
                        results.append({
                            'title': title,
                            'base_url': base_url,
                            'link': url
                        })
                        if len(results) >= max_results:
                            break
            tries += 1
            time.sleep(0.6)

        except Exception as e:
            print("Failed to scrape the page:", e)
            print("Retries left:", retries-tries)
            tries += 1
            time.sleep(0.6)

    if len(results) == 0:
        raise Exception(f"Max retries exceeded: {retries}")

    return results


In [ ]:
print("The Crawler has the following configuraions:")
print(conf)


keywords = conf["keywords"]

all_results = []

for keyword in keywords:
    print(f"Searching for keyword: {keyword}")
    try:
        results = duckduckgo_search(keyword)
        all_results.extend(results)  # add results for this keyword
    except Exception as e:
        print(f"Search failed for keyword '{keyword}': {e}")

url_list = [result['link'] for result in all_results]



crawler_threads = []
error_links = []

before_starting = datetime.now()

links_to_crawl=[]


for link in url_list:
    print(f"Scraping: {link}")
    scrape_and_save(link, conf, crawled_csv)
time.sleep(10)
run(conf['dir'] + "output.json", "summarize")

The Crawler has the following configuraions:
{'dir': '/content/drive/My Drive/KAIST/Spider1/', 'imgsDir': '/content/drive/My Drive/KAIST/Spider1/imgs/', 'keywords': ['Elon Musk', 'Tour Eiffel'], 'labels': ['technology', 'business', 'politics', 'history'], 'blacklist': ['youtube.com', 'facebook.com', 'twitter.com', 'x.com', 'instagram.com', 'linkedin.com', 'tiktok.com']}
Searching for keyword: Elon Musk
Searching for keyword: Tour Eiffel
Failed to scrape the page: No links found — possibly blocked or unexpected page structure.
Retries left: 99
Scraping: https://en.wikipedia.org/wiki/Elon_Musk
Done scraping https://en.wikipedia.org/wiki/Elon_Musk
Scraping: https://www.biography.com/business-leaders/elon-musk
Done scraping https://www.biography.com/business-leaders/elon-musk
Scraping: https://www.britannica.com/money/Elon-Musk
Done scraping https://www.britannica.com/money/Elon-Musk
Scraping: https://www.bbc.com/news/business-61234231
Done scraping https://www.bbc.com/news/business-612342

Done scraping https://mbeinstitute.org/ap/author-other/musk/elon-musk-biography-241106-0307.pdf
Scraping: https://www.toureiffel.paris/en
Done scraping https://www.toureiffel.paris/en
Scraping: https://en.wikipedia.org/wiki/Eiffel_Tower
Done scraping https://en.wikipedia.org/wiki/Eiffel_Tower
Scraping: https://fr.wikipedia.org/wiki/Tour_Eiffel
Done scraping https://fr.wikipedia.org/wiki/Tour_Eiffel
Scraping: https://www.britannica.com/topic/Eiffel-Tower-Paris-France
Done scraping https://www.britannica.com/topic/Eiffel-Tower-Paris-France
Scraping: https://ticket.toureiffel.paris/en
Done scraping https://ticket.toureiffel.paris/en
Scraping: https://thetourguy.com/travel-blog/france/paris/eiffel-tower/how-to-visit-this-year-tickets-hours-tours-and-more/
Done scraping https://thetourguy.com/travel-blog/france/paris/eiffel-tower/how-to-visit-this-year-tickets-hours-tours-and-more/
Scraping: https://www.city-paris.fr/en/visit-the-eiffel-tower/
Done scraping https://www.city-paris.fr/en/visi

Your max_length is set to 500, but your input_length is only 139. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)


finish
Top Predictions: technology
<15155CFB8377E44D80E04AA1EC8D7CB3>]/Index[22 28]/Info 21 0 R/Length 87/Prev 93382/Root 23 0 R/Size 50/Type/XRef/W[1 2 1]>>stream
h�bbd``b`�
$�� �
b��{ %� H/s9Ho:���H^�L�KA����~
 Streamers are using an index[22 28]/Info 21 0 R/Length 87/Prev 93382/Root 23 R/Size 50/Type/XRef/W[1 2 1] Streamers use a stream of data .


Your max_length is set to 500, but your input_length is only 188. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=94)


finish
Top Predictions: history
Come and discover the Eiffel Tower on the only trip to the top of its kind in Europe, and let pure emotions carry you from the esplanade to the top. Discover unique gifts, rare authentic collectibles and official products for unforgettable moments! Come and discover the ultimate immersive experience Enhance your experience at the Tower with your mobile phone. No download required! History, key figures, lights, paintings, explore all the secrets of the world's most iconic monument On every floor, the Eiffel Tower has a range of dining and gift-shop options. For your business events, hire a new venue in a magical setting. Fully equipped to meet all of your needs. The Eiffel Tower on social media Save time, buy your tickets online toureiffel.paris the official website of the Eiffel Tower. A website owned by Sociéte d'Exploitation de la tour Eiffel
 Come and discover the Eiffel Tower on the only trip to the top of its kind in Europe . Discover unique gifts, 

Your max_length is set to 500, but your input_length is only 11. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)


finish
Top Predictions: business
Please enable JavaScript to continue using this application.
 Please enable JavaScript to continue using this application . Use this article to help us understand today's featured news stories . Use the weekly Newsquiz to test your knowledge of stories you saw in the news .
finish
Top Predictions: technology
The Eiffel Tower: Top Tips for Visitors The Eiffel Tower: Top Tips for Visitors The Eiffel Tower: Top Tips for Visitors A reservation can help you avoid the Eiffel Tower's long lines. (photo: Rick Steves) Paris views are better from the tower's second level than its very top, as you're closer to the city's rooftops. (photo: Cathy Lu) is one of the great travel thrills in Europe. Sure, it's crowded and expensive, and other spots in Paris offer (arguably) better views. But once you make the eye-popping ascent — and ear-popping descent — you'll be in the exclusive society of some 250 million people who have made the Eiffel Tower one of the most visited